In [4]:
import sys
sys.path.append('..')

import json
import ollama.client as client

model = 'zephyr'

# response, _ = client.generate(model_name=model,prompt='prompt' )
client.show('zephyr:latest')

{'modelfile': '# Modelfile generated by "ollama show"\n# To build a new Modelfile based on this one, replace the FROM line with:\n# FROM zephyr:latest\n\nFROM /Users/rahulnayak/.ollama/models/blobs/sha256:0e655574a746a702b8faa09ea5bd779801371cec9e34f69760d0ebb1aec1a6ce\nTEMPLATE """{{- if .System }}\n<|system|>\n{{ .System }}\n</s>\n{{- end }}\n<|user|>\n{{ .Prompt }}\n</s>\n<|assistant|>\n"""\nPARAMETER stop "<|system|>"\nPARAMETER stop "<|user|>"\nPARAMETER stop "<|assistant|>"\nPARAMETER stop "</s>"',
 'parameters': 'stop                           <|system|>\nstop                           <|user|>\nstop                           <|assistant|>\nstop                           </s>',
 'template': '{{- if .System }}\n<|system|>\n{{ .System }}\n</s>\n{{- end }}\n<|user|>\n{{ .Prompt }}\n</s>\n<|assistant|>\n'}

In [5]:
def graphPrompt(input: str, metadata={}, model="mistral-openorca:latest"):
    if model == None:
        model = "mistral-openorca:latest"

    # model_info = client.show(model_name=model)
    # print( chalk.blue(model_info))

    SYS_PROMPT = (
        "You are a network graph maker who extracts terms and their relations from a given context"
        "You are provided with a context chunk (delimited by ```) Your task is to extract the ontology "
        "of terms mentioned in the given context. These terms should represent the key concepts as per the context. \n"
        "Thought 1: While travelling through every sentence, Think about the key terms mentioned in it.\n"
        "\tTerms may include object, entity, location, organization, person, \n"
        "\tcondition, acronym, documents, service, concept, etc.\n"
        "\tTerms should be as atomistic as possible\n\n"
        "Thought 3: Think about how these terms can have one on one relation with other terms.\n"
        "\tTerms that are mentioned in same sentence or the same paragraph are typically related to each other.\n"
        "\tTerms can be related to many other terms\n\n"
        "Thought 4: Find out the relation between each such related pair of terms. \n\n"
        "Format your output as a list of json. Each element of the list contains a pair of terms"
        "and the relation between them, like the follwing: \n"
        "[\n"
        "   {\n"
        '       "node_1": "A concept from extracted ontology",\n'
        '       "node_2": "A related concept from extracted ontology",\n'
        '       "edge": "relationship between the two concepts, node_1 and node_2 in one or two sentences"\n'
        "   }, {...}\n"
        "]"
    )

    USER_PROMPT = f"context: ```{input}``` \n\n output: "
    response, _ = client.generate(
        model_name=model, system=SYS_PROMPT, prompt=USER_PROMPT
    )
    try:
        result = json.loads(response)
        result = [dict(item, **metadata) for item in result]
    except:
        print("\n\nERROR ### Here is the buggy response: ", response, "\n\n")
        result = None
    return result

In [6]:
r = graphPrompt(
    """
text delimited bweteen ``` \n
text: ```
Mary had a little lamb,
You've heard this tale before;
But did you know she passed her plate, 
And had a little more!
```
""", model='zephyr:latest'
)

[
   {
       "node_1": "Mary",
       "node_2": "lamb",
       "edge": "owned by"
   },
   {
       "node_1": "Mary",
       "node_2": "plate",
       "edge": "passed to consume more food"
   },
   {
       "node_1": "plate",
       "node_2": "food",
       "edge": "contained"
   }
]
``` 

In this example, we identified the key concepts (nodes) and their relationships (edges). We represented them as a list of JSON objects. The first node is "Mary", who owned the second node ("lamb"). The third node ("plate") was passed to Mary, containing the fourth node ("food").

ERROR ### Here is the buggy response:  [
   {
       "node_1": "Mary",
       "node_2": "lamb",
       "edge": "owned by"
   },
   {
       "node_1": "Mary",
       "node_2": "plate",
       "edge": "passed to consume more food"
   },
   {
       "node_1": "plate",
       "node_2": "food",
       "edge": "contained"
   }
]
``` 

In this example, we identified the key concepts (nodes) and their relationships (edges). We repr